In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('protein-optimization/sc_diff/model-ydqtt928:v12', type='model')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnyang (protein-optimization). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [2]:
artifact_dir

'./artifacts/model-ydqtt928:v12'

In [3]:
% cd artifact_dir
% ls

UsageError: Line magic function `%` not found.


In [4]:
from cellot.models.cond_score_module import CondScoreModule

In [5]:
ckpt_path = f'{artifact_dir}/model.ckpt'

In [6]:
YAML_STR = '''
DEBUG: False
TARGET: all
LATENT_DIM: 50
COND_CLASSES: 189
SEED: 42
AE_PATH: /Mounts/rbg-storage1/users/johnyang/cellot/results/sciplex3/full_ae
VAL_SIZE: 0.1
DEVICES: 1

diffuser:
  min_b: 0.01
  max_b: 1.0
  schedule: linear
  score_scaling: var
  coordinate_scaling: 1.0
  latent_dim: ${LATENT_DIM}
  dt: 0.01
  min_t: 0

ae:
  name: scgen
  beta: 0.0
  dropout: 0.0
  hidden_units: [512, 512]
  latent_dim: 50

score_network:
  latent_dim: ${LATENT_DIM}
  cond_classes: ${COND_CLASSES}
  model_dim: 64   # Adjusted to 64
  n_layers: 12    # Adjusted to 12
  nhead: 8
  dim_feedforward: 2048
  dropout: 0.1
  ffn_hidden_dim: 1024


data:
  type: cell
  source: control
  condition: drug
  path: /Mounts/rbg-storage1/users/johnyang/cellot/datasets/scrna-sciplex3/hvg.h5ad
  target: ${TARGET}

datasplit:
  groupby: drug   
  name: train_test
  test_size: 0.2
  random_state: 0
  
dataloader:
  batch_size: 256   # Adjusted to 256
  shuffle: true
  num_workers: 80
  
experiment:
  name: base
  mode: train
  num_loader_workers: 0
  port: 12319
  dist_mode: single
  use_wandb: True
  ckpt_path: null
  wandb_logger:
    project: sc_diff
    name: ${experiment.name}
    dir: /Mounts/rbg-storage1/users/johnyang/cellot/
    log_model: all
    tags: ['experimental']
  lr: 0.0001


trainer:
  accelerator: 'gpu'
  check_val_every_n_epoch: 50
  log_every_n_steps: 100
  num_sanity_val_steps: 1
  enable_progress_bar: True
  enable_checkpointing: True
  fast_dev_run: False
  profiler: simple
  max_epochs: 10000
  strategy: auto
  enable_model_summary: True
  overfit_batches: 0.0
  limit_train_batches: 1.0
  limit_val_batches: 1.0
  limit_predict_batches: 1.0
'''

In [7]:
from omegaconf import OmegaConf
config = OmegaConf.create(YAML_STR)

In [8]:
lm = CondScoreModule.load_from_checkpoint(hparams=config, checkpoint_path=ckpt_path)

Dropout is 0.1


In [9]:
from cellot.train.utils import get_free_gpu
replica_id = int(get_free_gpu())

cuda
Using GPUs: 0


In [10]:
from cellot.data.sciplex_ae_dm import CellDataModule
dm = CellDataModule(config)

In [11]:
dm.setup('fit')

Loading ae from checkpoint


In [12]:
device = f'cuda:{replica_id}'

In [13]:
lm.to(device)
print('')

In [14]:
for batch in dm.val_dataloader():
    batch = [b.to(device) for b in batch]
    mse = lm.validation_step(batch, None)
    break

/data/rsg/chemistry/johnyang/miniconda3/envs/cellot/lib/python3.9/site-packages/pytorch_lightning/core/module.py:407: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


In [15]:
mse

tensor(0.8200, device='cuda:0', dtype=torch.float64)

In [16]:
from pytorch_lightning import Trainer
trainer_devices = [replica_id]
trainer = Trainer(devices=trainer_devices, **config.trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_predict_batches=1.0)` was configured so 100% of the batches will be used..


In [17]:
import os

In [18]:
# Run ID for your WandB run
run_id = 'ydqtt928'  # replace with your run ID

# Names of the artifacts in the run
artifact_names = [f'model-ydqtt928:v{num}' for num in range(0, 13, 4)]  # replace with your artifact names

run = wandb.init(project='sc_diff', entity='protein-optimization')
# run_name = "protein-optimization/sc_diff/"

In [19]:
val_metrics = []
# Loop over each artifact, download the weights, load into the model, and validate
for artifact_name in artifact_names:
    artifact = run.use_artifact(artifact_name, type='model')

    artifact_dir = artifact.download()
    ckpt_path = f'{artifact_dir}/model.ckpt'
    # Load the model from the checkpoint
    model = CondScoreModule.load_from_checkpoint(checkpoint_path=ckpt_path, hparams=config)

    # Move the model to the right device
    model.to(device)

    # Run validation
    metrics = trainer.validate(model, datamodule=dm, verbose=True)
    val_metrics.append(metrics)

wandb:   1 of 1 files downloaded.  


Dropout is 0.1
Loading ae from checkpoint


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Validation DataLoader 0: 100%|██████████| 239/239 [06:18<00:00,  1.58s/it]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val/mse            0.8834621906280518
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


VALIDATE Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                          	|  -              	|  3381        

...

VALIDATE Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                          	|  -              	|  10143       

Dropout is 0.1
Loading ae from checkpoint


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Validation DataLoader 0: 100%|██████████| 239/239 [05:29<00:00,  1.38s/it]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val/mse            0.8036770224571228
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


VALIDATE Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                          	|  -              	|  13524       